<a href="https://colab.research.google.com/github/ayushanand960/Age_Detection_using_UTK_Face_Dataset/blob/main/age_detection_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Mount Google Drive**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Importing Libraries**


In [2]:
import zipfile
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

**Check For GPU**

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


**Importing Dataset**

In [4]:
zip_file_path = '/content/drive/MyDrive/UTKFace_Dataset.zip'  # Change this to your zip file path
extract_path = '/content/UTKFace'
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

**Extracting DataSet**

In [5]:
data_dir = extract_path  # Use the extracted path
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='sparse',  # Use 'sparse' for age regression
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(200, 200),
    batch_size=32,
    class_mode='sparse',  # Use 'sparse' for age regression
    subset='validation'
)

Found 53582 images belonging to 3 classes.
Found 13394 images belonging to 3 classes.


**Importing Pre-trained Model**

In [6]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(200, 200, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


# CNN Architecture

In [7]:
x = base_model.output
x = Flatten()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
predictions = Dense(1, activation='linear')(x)  # For age prediction
model = Model(inputs=base_model.input, outputs=predictions)


In [8]:
model.compile(optimizer=Adam(learning_rate=1e-4), loss='mean_squared_error', metrics=['mae'])

In [9]:
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=15,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Epoch 1/15


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


1675/1675 ━━━━━━━━━━━━━━━━━━━━ 867s 493ms/step - loss: 0.9467 - mae: 0.8791 - val_loss: 0.8765 - val_mae: 0.8778
Epoch 2/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 0s 17us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 3/15


/usr/lib/python3.11/contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


1675/1675 ━━━━━━━━━━━━━━━━━━━━ 782s 467ms/step - loss: 0.8455 - mae: 0.8527 - val_loss: 0.8613 - val_mae: 0.8720
Epoch 4/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 5/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 779s 465ms/step - loss: 0.8368 - mae: 0.8508 - val_loss: 0.9165 - val_mae: 0.8903
Epoch 6/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 34s 20ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 7/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 826s 464ms/step - loss: 0.8373 - mae: 0.8514 - val_loss: 0.8742 - val_mae: 0.8764
Epoch 8/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 34s 21ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 9/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 824s 462ms/step - loss: 0.8357 - mae: 0.8514 - val_loss: 0.9113 - val_mae: 0.8903
Epoch 10/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 11/15
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 798s 459ms/step - loss: 0.8334 - mae: 0.8513 - val_loss: 0.8717 - val_mae: 0.8773
Epoch 12/15
1675/16

# Fine Tunning pre-trained CNN model

In [10]:
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers
    layer.trainable = True

model.compile(optimizer=Adam(learning_rate=1e-5), loss='mean_squared_error', metrics=['mae'])
history_fine = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    steps_per_epoch=len(train_generator),
    validation_steps=len(validation_generator)
)


Epoch 1/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 773s 457ms/step - loss: 0.8250 - mae: 0.8466 - val_loss: 0.9011 - val_mae: 0.8894
Epoch 2/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 35s 21ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 3/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 763s 455ms/step - loss: 0.8237 - mae: 0.8465 - val_loss: 0.9009 - val_mae: 0.8905
Epoch 4/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 0s 7us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 5/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 763s 456ms/step - loss: 0.8211 - mae: 0.8456 - val_loss: 0.9054 - val_mae: 0.8923
Epoch 6/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 0s 8us/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 7/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 763s 455ms/step - loss: 0.8223 - mae: 0.8448 - val_loss: 0.9105 - val_mae: 0.8943
Epoch 8/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 35s 21ms/step - loss: 0.0000e+00 - mae: 0.0000e+00
Epoch 9/20
1675/1675 ━━━━━━━━━━━━━━━━━━━━ 768s 456ms/step - loss: 0.8249 - mae: 0.8469 - val_loss: 0.9118 - val_mae: 0.8947
Epoch 10/20


**Checking accuracy**

In [11]:
test_loss, test_mae = model.evaluate(validation_generator)
print(f'Test MAE: {test_mae}')

419/419 ━━━━━━━━━━━━━━━━━━━━ 50s 118ms/step - loss: 0.9177 - mae: 0.8983
Test MAE: 0.8972932696342468


In [13]:
test_loss, test_accuracy = model.evaluate(validation_generator)
print(f'Test Accuracy: {test_accuracy:.2f}')

419/419 ━━━━━━━━━━━━━━━━━━━━ 50s 119ms/step - loss: 0.9225 - mae: 0.9014
Test Accuracy: 0.90


**Saving Model**

In [14]:
model.save('/content/drive/MyDrive/age_detection_model.h5')  # Change this to your desired save path